# Star Wars data
All data from http://swapi.co using their python API.

In [2]:
import swapi
import cPickle

In [17]:
class Slurper(object):
    def __init__(self, category):
        self.container = swapi.get_all(category)
        self.data = list(self.container.iter())
        self.records = None
        
    def set_records(self, records):
        self.records = [tuple(self.fix_entry(s) for s in rec) for rec in records]
    
    def export_csv(self, fn):
        with open(fn, "w") as out:
            for rec in self.records:
                out.write(",".join(rec) + '\n')
    
    def fix_entry(self, entry):
        return entry.replace(",", "").strip().encode('ascii', 'ignore')


## Planets 

In [18]:
planets = Slurper("planets")

In [19]:
planets.set_records((p.name, p.diameter, p.terrain) for p in planets.data)

In [22]:
planets.export_csv("planets.csv")

## People 

In [63]:
people = swapi.get_all('people')

In [64]:
p_list = [p for p in people.iter()]

In [65]:
x = p_list[0]

In [66]:
x.get_homeworld().name

u'Tatooine'

In [67]:
x.name

u'Luke Skywalker'

In [68]:
x.eye_color

u'blue'

In [69]:
x.birth_year

u'19BBY'

In [70]:
x.get_starships()

<StarshipQuerySet - 2>

In [71]:
records = [(p.name, p.get_homeworld().name, p.eye_color, p.mass) for p in p_list]

In [72]:
records

[(u'Luke Skywalker', u'Tatooine', u'blue', u'77'),
 (u'C-3PO', u'Tatooine', u'yellow', u'75'),
 (u'R2-D2', u'Naboo', u'red', u'32'),
 (u'Darth Vader', u'Tatooine', u'yellow', u'136'),
 (u'Leia Organa', u'Alderaan', u'brown', u'49'),
 (u'Owen Lars', u'Tatooine', u'blue', u'120'),
 (u'Beru Whitesun lars', u'Tatooine', u'blue', u'75'),
 (u'R5-D4', u'Tatooine', u'red', u'32'),
 (u'Biggs Darklighter', u'Tatooine', u'brown', u'84'),
 (u'Obi-Wan Kenobi', u'Stewjon', u'blue-gray', u'77'),
 (u'Anakin Skywalker', u'Tatooine', u'blue', u'84'),
 (u'Wilhuff Tarkin', u'Eriadu', u'blue', u'unknown'),
 (u'Chewbacca', u'Kashyyyk', u'blue', u'112'),
 (u'Han Solo', u'Corellia', u'brown', u'80'),
 (u'Greedo', u'Rodia', u'black', u'74'),
 (u'Jabba Desilijic Tiure', u'Nal Hutta', u'orange', u'1,358'),
 (u'Wedge Antilles', u'Corellia', u'hazel', u'77'),
 (u'Jek Tono Porkins', u'Bestine IV', u'blue', u'110'),
 (u'Yoda', u'unknown', u'brown', u'17'),
 (u'Palpatine', u'Naboo', u'yellow', u'75'),
 (u'Boba Fett',

In [73]:
len(records)

87

In [74]:
cPickle.dump(records, open("people.pkl", "w"), protocol=cPickle.HIGHEST_PROTOCOL)

In [75]:
with open("people.csv", "w") as out:
    for rec in records:
        out.write(",".join([s.encode('ascii', 'ignore') for s in rec]) + '\n')

## Vehicles 

In [52]:
vehicles = swapi.get_all('vehicles')

In [53]:
v_list = [p for p in vehicles.iter()]

In [54]:
v = v_list[0]

In [55]:
v.name

u'Sand Crawler'

In [56]:
v.model

u'Digger Crawler'

In [57]:
v.vehicle_class

u'wheeled'

In [58]:
v.cost_in_credits

u'150000'

In [59]:
v.max_atmosphering_speed

u'30'

In [86]:
v_records = [(v.name, v.model, v.vehicle_class, v.max_atmosphering_speed, v.cost_in_credits) for v in v_list]

In [87]:
cPickle.dump(v_records, open("vehicles.pkl", "w"), protocol=cPickle.HIGHEST_PROTOCOL)

In [88]:
with open("vehicles.csv", "w") as out:
    for rec in v_records:
        out.write(",".join([s.encode('ascii', 'ignore') for s in rec]) + '\n')

## Starships 

In [40]:
ships = swapi.get_all('starships')

In [41]:
s_list = [p for p in ships.iter()]

In [42]:
s_list

[<Starship - Sentinel-class landing craft>,
 <Starship - Death Star>,
 <Starship - Millennium Falcon>,
 <Starship - Y-wing>,
 <Starship - X-wing>,
 <Starship - TIE Advanced x1>,
 <Starship - Executor>,
 <Starship - Slave 1>,
 <Starship - Imperial shuttle>,
 <Starship - EF76 Nebulon-B escort frigate>,
 <Starship - Calamari Cruiser>,
 <Starship - A-wing>,
 <Starship - B-wing>,
 <Starship - Republic Cruiser>,
 <Starship - Naboo fighter>,
 <Starship - Naboo Royal Starship>,
 <Starship - Scimitar>,
 <Starship - J-type diplomatic barge>,
 <Starship - AA-9 Coruscant freighter>,
 <Starship - Jedi starfighter>,
 <Starship - H-type Nubian yacht>,
 <Starship - Star Destroyer>,
 <Starship - Trade Federation cruiser>,
 <Starship - Theta-class T-2c shuttle>,
 <Starship - T-70 X-wing fighter>,
 <Starship - Rebel transport>,
 <Starship - Droid control ship>,
 <Starship - Republic Assault ship>,
 <Starship - Solar Sailer>,
 <Starship - Republic attack cruiser>,
 <Starship - Naboo star skiff>,
 <Starshi

In [76]:
s = s_list[2]

In [77]:
s.name

u'Millennium Falcon'

In [78]:
s.cost_in_credits

u'100000'

In [79]:
s.length

u'34.37'

In [80]:
s.max_atmosphering_speed

u'1050'

In [81]:
s.starship_class

u'Light freighter'

In [82]:
s.hyperdrive_rating

u'0.5'

In [83]:
s.model

u'YT-1300 light freighter'

In [89]:
s_records = [(v.name, v.model, v.starship_class, v.max_atmosphering_speed, v.cost_in_credits, v.length) for v in s_list]

In [90]:
cPickle.dump(s_records, open("starships.pkl", "w"), protocol=cPickle.HIGHEST_PROTOCOL)

In [91]:
with open("starships.csv", "w") as out:
    for rec in s_records:
        out.write(",".join([s.encode('ascii', 'ignore') for s in rec]) + '\n')